In [1]:
![ -d ./sample_data/ ] && rm -rf ./sample_data/

In [2]:
!pip install transformers &>/dev/null

In [3]:
import transformers

### Using pre-trained transformers (2pts)
_for fun and profit_

There are many toolkits that let you access pre-trained transformer models, but the most powerful and convenient by far is [`huggingface/transformers`](https://github.com/huggingface/transformers). In this week's practice, you'll learn how to download, apply and modify pre-trained transformers for a range of tasks. Buckle up, we're going in!


__Pipelines:__ if all you want is to apply a pre-trained model, you can do that in one line of code using pipeline. Huggingface/transformers has a selection of pre-configured pipelines for masked language modelling, sentiment classification, question aswering, etc. ([see full list here](https://huggingface.co/transformers/main_classes/pipelines.html))

A typical pipeline includes:
* pre-processing, e.g. tokenization, subword segmentation
* a backbone model, e.g. bert finetuned for classification
* output post-processing

Let's see it in action:

In [4]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

print(classifier("BERT is amazing!"))

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998860359191895}]


In [5]:
import base64
data = {
    'arryn': 'As High as Honor.',
    'baratheon': 'Ours is the fury.',
    'stark': 'Winter is coming.',
    'tyrell': 'Growing strong.'
}

# YOUR CODE: predict sentiment for each noble house and create outputs dict
# <...>
# outputs = <YOUR CODE: dict (house name) : True if positive, False if negative>

outputs={i : True if classifier(data[i])[0]['label']=="POSITIVE" else False for i in list(data.keys())}

assert sum(outputs.values()) == 3 and outputs[base64.decodestring(b'YmFyYXRoZW9u\n').decode()] == False
print("Well done!")

Well done!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()
  from ipykernel import kernelapp as app


You can also access vanilla Masked Language Model that was trained to predict masked words. Here's how:

In [6]:
mlm_model = pipeline('fill-mask', model="bert-base-uncased")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"Donald {MASK} is the president of the united states."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

P=0.99719 donald trump is the president of the united states.
P=0.00024 donald duck is the president of the united states.
P=0.00022 donald ross is the president of the united states.
P=0.00020 donald johnson is the president of the united states.
P=0.00018 donald wilson is the president of the united states.


In [7]:
# Your turn: use bert to recall what year was the Soviet Union founded in
# mlm_model(<YOUR PROMPT>)

for hypo in mlm_model (f"In the year of {MASK} the Soviet Union was established."):
  print (f"P={hypo['score']:.5f}", hypo['sequence'])

P=0.08584 in the year of 1941 the soviet union was established.
P=0.06727 in the year of 1945 the soviet union was established.
P=0.06630 in the year of 1940 the soviet union was established.
P=0.05883 in the year of 1917 the soviet union was established.
P=0.04899 in the year of 1918 the soviet union was established.


```

```

```

```


Huggingface offers hundreds of pre-trained models that specialize on different tasks. You can quickly find the model you need using [this list](https://huggingface.co/models).


In [8]:
text = """Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
 the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
 Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet 
 for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
  “We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""

# Task: create a pipeline for named entity recognition, use task name 'ner' and search for the right model in the list
# ner_model = <YOUR CODE>
ner_model=pipeline ("ner", model="dslim/bert-base-NER")

named_entities = ner_model(text)

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
print('OUTPUT:', named_entities)
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

OUTPUT: [{'entity': 'B-LOC', 'score': 0.7991051, 'index': 27, 'word': 'Rose', 'start': 112, 'end': 116}, {'entity': 'I-LOC', 'score': 0.9511928, 'index': 28, 'word': '##tta', 'start': 116, 'end': 119}, {'entity': 'B-ORG', 'score': 0.998223, 'index': 40, 'word': 'Guardian', 'start': 179, 'end': 187}, {'entity': 'B-PER', 'score': 0.9997613, 'index': 46, 'word': 'Ian', 'start': 207, 'end': 210}, {'entity': 'I-PER', 'score': 0.99978703, 'index': 47, 'word': 'Sam', 'start': 211, 'end': 214}, {'entity': 'I-PER', 'score': 0.999646, 'index': 48, 'word': '##ple', 'start': 214, 'end': 217}, {'entity': 'B-PER', 'score': 0.9997831, 'index': 53, 'word': 'Stuart', 'start': 240, 'end': 246}, {'entity': 'I-PER', 'score': 0.99974823, 'index': 54, 'word': 'Clark', 'start': 247, 'end': 252}, {'entity': 'B-LOC', 'score': 0.9997227, 'index': 85, 'word': 'Germany', 'start': 414, 'end': 421}, {'entity': 'B-PER', 'score': 0.9963128, 'index': 99, 'word': 'Phil', 'start': 471, 'end': 475}, {'entity': 'I-PER', '

### The building blocks of a pipeline

Huggingface also allows you to access its pipelines on a lower level. There are two main abstractions for you:
* `Tokenizer` - converts from strings to token ids and back
* `Model` - a pytorch `nn.Module` with pre-trained weights

You can use such models as part of your regular pytorch code: insert is as a layer in your model, apply it to a batch of data, backpropagate, optimize, etc.

In [10]:
import torch, torch.nn as nn
from transformers import AutoTokenizer, AutoModel, pipeline
from torch.autograd import Variable

In [11]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# tokenize a batch of inputs. "pt" means [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

for key in tokens_info:
    print(key, tokens_info[key])

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

input_ids tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Detokenized:
[CLS] luke, i am your father. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] life is what happens when you're busy making other plans. [SEP]


In [13]:
# You can now apply the model to get embeddings
with torch.no_grad():
    out = model(**tokens_info)

print(out)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3502,  0.2246, -0.2345,  ..., -0.2232,  0.1730,  0.6747],
         [-0.6097,  0.6892, -0.5512,  ..., -0.4814,  0.5322,  1.3833],
         [ 0.1842,  0.4881,  0.2193,  ..., -0.2699,  0.2246,  0.7985],
         ...,
         [-0.4413,  0.2748, -0.0391,  ..., -0.0604, -0.4358,  0.1384],
         [-0.5414,  0.4633,  0.0678,  ..., -0.1871, -0.5046,  0.2752],
         [-0.3940,  0.6180,  0.2092,  ..., -0.2345, -0.4177,  0.3341]],

        [[ 0.1622, -0.1154, -0.3894,  ..., -0.4180,  0.0138,  0.7644],
         [ 0.6471,  0.3774, -0.4082,  ...,  0.0050,  0.5559,  0.4385],
         [ 0.3351, -0.3158, -0.1178,  ...,  0.1348, -0.3143,  1.4409],
         ...,
         [ 1.2932, -0.1743, -0.5613,  ..., -0.2718, -0.1367,  0.4217],
         [ 1.0304,  0.1708, -0.2985,  ...,  0.2097, -0.4627, -0.4277],
         [ 1.0854,  0.1760, -0.0377,  ...,  0.3152, -0.5979, -0.3465]]]), pooler_output=tensor([[-0.8854, -0.4722, -0.9392,  .

### Fine-tuning for salary prediction (5 pts) => For this part see the second notebook

Now let's put all this monstrosity to good use!

Remember week5 when you've trained a convolutional neural network for salary prediction? Now let's see how transformers fare at this task.

__The goal__ is to take one or more pre-trained models and fine-tune it for salary prediction. A good baseline solution would be to get RoBerta or T5 from [huggingface model list](https://huggingface.co/models) and fine-tune it to solve the task. After choosing the model, please take care to use the matching Tokenizer for preprocessing, as different models have different preprocessing requirements.


There are no prompts this time: you will have to write everything from scratch. Although, feel free to reuse any code from the original salary prediction notebook :)

In [14]:
# !wget https://www.dropbox.com/s/r9d1f3ve471osob/Train_rev1.zip?dl=1 -O data.zip &> /dev/null
# !unzip -e data.zip &>/dev/null

In [15]:
# <A whole lot of your code. Feel free to format it as you see fit>

'''
For this part see the next notebook

'''

'\nFor this part see the next notebook\n\n'

### The search for similar questions (3pts)

* Implement a function that takes a text string and finds top-k most similar questions from `quora.txt`
* Demonstrate your function using at least 5 examples

There are no prompts this time: you will have to write everything from scratch.


In [16]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2021-11-12 16:08:33--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2021-11-12 16:08:33--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde33908728ed7111915d1c9262.dl.dropboxusercontent.com/cd/0/get/BZ1aEt8-sfOH18xfASgv7k3JVKVjGHfSY7WOm-fyRUs_dvAJtFv94cLkUv5L9k1EOa17VyYOicNM3pEN2l8ECP_eLJLMT6dWTFLlHP6uCX0gNs1hIYv6ld-jqxYMsxjjei3q5pB1fR_xBR3kalM8GY0A/file?dl=1# [following]
--2021-11-12 16:08:33--  https://ucde33908728ed7111915d1c9262.dl.dropboxusercontent.com/cd/0/get/BZ1aEt8-sfOH18xfASgv7k3JVKVjGHfSY7WOm-fyRUs_dvAJtFv94cLkUv5L9k1EOa17VyYOicNM3pEN2l8ECP_eLJLMT6

In [33]:
# <A whole lot of your code. Feel free to format it as you see fit>

import pandas as pd

quora=pd.read_fwf ("quora.txt", header=None)
quora.fillna ("unknown", inplace=True)

In [34]:
quora.values.reshape(-1)

array(["Can I get back with my ex even though she is pregnant with another guy's baby?",
       'What are some ways to overcome a fast food addiction?',
       'Who were the great Chinese soldiers and leaders who fought in WW2?',
       ...,
       'Is it necessary to do a diploma in nautical science before joining IMU?',
       'Can you debeak cockerels at 8 months old?',
       'What are the most unintentionally hilarious movies you have ever seen?'],
      dtype=object)

In [66]:
quest=quora.values.reshape(-1).tolist()

In [134]:
!pip install -q sentence_transformers

     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 1.2 MB 8.3 MB/s 


In [141]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('paraphrase-MiniLM-L6-v2').to('cuda')

In [142]:
embeds=model.encode(quest)

In [160]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [197]:
def find_similar (text=text, topk=5, model=model, embed=embeds):
  embed_text=model.encode(text)
  sim=cosine_similarity(embed_text.reshape(1, -1), embed)

  return sim.argsort()[0][::-1][:5].tolist()

In [213]:
text=[
      "Can I be together with my previous girlfriend who is expecting a child?",
      "How to learn to play piano?",
      "I think I am loosing my mind, what should I do?",
      "Is reincarnation real? If yes, how to know who were you in your previous life?",
      "Is Elon Musk really genius, or he is just lucky?", 
      "Can I breathe in space?",
      "To be or not to be?",
      "Mister my friend, how are you?",
      "Dowak aamk ii?"]

In [214]:
for i in text:
  print ("The query: ", i)
  print ("Similart queries: \n")
  sim=find_similar(i)
  for j in sim:
    print (quest[j])
  print ("\n")

The query:  Can I be together with my previous girlfriend who is expecting a child?
Similart queries: 

What should I do if my girlfriend is pregnant?
Can I get back with my ex even though she is pregnant with another guy's baby?
I love her and she loves me, but she doesn't see the possibility of us being together in the future. What do I do?
I am in love with a girl who is married to other guy. Should we continue?
My girlfriend is 2 months pregnant and we dont want to have a baby just yet. What to do?


The query:  How to learn to play piano?
Similart queries: 

How do I learn play piano?
How to learn piano?
How can I learn how to play the piano?
What is the best way to learn to play piano?
Is there a way I could learn to play the piano?


The query:  I think I am loosing my mind, what should I do?
Similart queries: 

I want to destress my mind . Help me?
What should I do to control my mind?
How do I control my mind?
How do I control my thoughts and mind?
How can I control my mind fro

```















```

__Bonus demo:__ transformer language models. 

`/* No points awarded for this task, but its really cool, we promise :) */`

In [ ]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', add_prefix_space=True)
model = GPT2LMHeadModel.from_pretrained('gpt2').train(False).to(device)

text = "The Fermi paradox "
tokens = tokenizer.encode(text)
num_steps = 1024
line_length, max_length = 0, 70

print(end=tokenizer.decode(tokens))

for i in range(num_steps):
    with torch.no_grad():
        logits = model(torch.as_tensor([tokens], device=device))[0]
    p_next = torch.softmax(logits[0, -1, :], dim=-1).data.cpu().numpy()

    next_token_index = p_next.argmax() #<YOUR CODE: REPLACE THIS LINE>
    # YOUR TASK: change the code so that it performs nucleus sampling

    tokens.append(int(next_token_index))
    print(end=tokenizer.decode(tokens[-1]))
    line_length += len(tokenizer.decode(tokens[-1]))
    if line_length >= max_length:
        line_length = 0
        print()



Transformers knowledge hub: https://huggingface.co/transformers/